In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

C:\Users\Dinesh\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Loading Keras libraries
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
#Loading Fashion MNIST 
data = pd.read_csv('fashion-mnist_train.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [5]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
target = data['label'] # Target output variable
y = to_categorical(target)
y.shape

(60000, 10)

In [7]:
feature = data.iloc[:,1:] # Indexing Rows and columns

In [8]:
feature.shape

(60000, 784)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(feature, y, test_size=0.2, random_state=1) #Splitting data to train and test

In [29]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1) #Splitting train data to validation

In [30]:
x_train.shape

(38400, 784)

In [31]:
x_val.shape

(9600, 784)

In [32]:
x_test.shape
x_train = x_train.as_matrix()
x_test = x_test.as_matrix()
x_val = x_val.as_matrix()

In [33]:
#Changing image numpy array to 28*28 grayscale image from 784
img_rows = 28
img_cols = 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)

In [34]:
#Convert each image pixel size from 0 to 255 to 0 to 1
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255
x_val = x_val.astype('float32')/255

In [35]:
x_train.shape

(38400, 28, 28, 1)

In [36]:
x_test.shape

(12000, 28, 28, 1)

In [37]:
x_val.shape

(9600, 28, 28, 1)

In [38]:
# network
model = Sequential()
model.add(Conv2D(32, (5, 5), padding='same',input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [39]:
NB_CLASSES = 10
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
activation_4 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               3211776   
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
__________

In [40]:
#constant
BATCH_SIZE = 128
NB_EPOCH = 5
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

In [42]:
from keras.callbacks import EarlyStopping, TensorBoard
callbacks = [EarlyStopping(monitor='val_acc', patience=3), TensorBoard(log_dir='new-logs')]
x_train.shape
x_val.shape

(9600, 28, 28, 1)

In [48]:
# train
model.compile(loss='categorical_crossentropy', optimizer='adam',
metrics=['accuracy'])
model.fit(x_train, y_train, batch_size = 64, epochs=10)
#model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=0.2, callbacks=callbacks)
#model.fit(x_train, y_train, batch_size=BATCH_SIZE,
#epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT)

#batch_size=BATCH_SIZE)
#print("Test score:", score[0])
#print('Test accuracy:', score[1])

Epoch 1/10
38400/38400 [==============================] - 131s 3ms/step - loss: 0.2786 - acc: 0.8998
Epoch 2/10
38400/38400 [==============================] - 130s 3ms/step - loss: 0.2468 - acc: 0.9082
Epoch 3/10
38400/38400 [==============================] - 131s 3ms/step - loss: 0.2179 - acc: 0.9198
Epoch 4/10
38400/38400 [==============================] - 128s 3ms/step - loss: 0.1988 - acc: 0.9260
Epoch 5/10
38400/38400 [==============================] - 123s 3ms/step - loss: 0.1807 - acc: 0.9321
Epoch 6/10
38400/38400 [==============================] - 12472s 325ms/step - loss: 0.1663 - acc: 0.9373
Epoch 7/10
38400/38400 [==============================] - 86s 2ms/step - loss: 0.1512 - acc: 0.9423
Epoch 8/10
38400/38400 [==============================] - 86s 2ms/step - loss: 0.1412 - acc: 0.9474
Epoch 9/10
38400/38400 [==============================] - 89s 2ms/step - loss: 0.1297 - acc: 0.9513
Epoch 10/10
38400/38400 [==============================] - 90s 2ms/step - loss: 0.1216 - a

In [44]:
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

12000/12000 [==============================] - 8s 679us/step
Test score: 0.3233091001113256
Test accuracy: 0.8824999998410543


In [ ]:
opt = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
metrics=['accuracy'])

In [ ]:
model = Sequential([
    
])